First, we'll download a 1/3-arcsecond DEM of the region we're interested in.
The DEM comes from the Oregon Lidar Consortium, which is part of the Oregon Department of Geology and Mineral Industries (DOGAMI).
The DOGAMI website has an interactive online [viewer](https://gis.dogami.oregon.gov/maps/lidarviewer/) and download utility for LiDAR data.
The code below uses a library called [pooch](https://www.fatiando.org/pooch) to describe what file we want to get and from where.

In [ ]:
import pooch

url = "https://www.oregongeology.org/pubs/ldq/"
archive_filename = "LDQ-43124D1.zip"
checksum = "cb1fcb26fbb6e84640a554fb2287c619cfe6f54bc81a6423624273ceb21f7647"
dem = pooch.create(
    path=pooch.os_cache("hillslope"),
    base_url=url,
    registry={archive_filename: checksum},
)

Next we'll actually fetch the raw data, unzip it, and extract a `.adf` file (an ArcInfo binary format) containing the actual DEM.

In [ ]:
import requests

try:
    downloader = pooch.HTTPDownloader(progressbar=True)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )
except requests.exceptions.SSLError:
    downloader = pooch.HTTPDownloader(progressbar=True, verify=False)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )

In [ ]:
filename = [
    f for f in files if "South Coast" in f and "Bare_Earth" in f and "w001001.adf" in f
][0]

print(filename)

Now we can open the DEM using the library [rasterio](https://rasterio.readthedocs.io/en/latest/).
The original data use the EPSG:3644 coordinate system which is specialized to Oregon.
This CRS measures distance in feet, which is truly offensive, so we'll reproject to UTM zone 10 (EPSG:32610) immediately.

In [ ]:
import numpy as np
import numpy.ma as ma
import rasterio
import rasterio.warp

with rasterio.open(filename, "r") as source:
    elevation, transform = rasterio.warp.reproject(
        source=source.read(indexes=1),
        src_transform=source.transform,
        src_crs="EPSG:3644",
        dst_crs="EPSG:32610",
        resampling=rasterio.enums.Resampling.bilinear,
    )

    bounds = source.bounds
    feet_to_meters = 0.3048
    elevation = feet_to_meters * ma.masked_less_equal(elevation[0, :, :], 0.0)

In [ ]:
import matplotlib.pyplot as plt

extent = rasterio.warp.transform_bounds(
    src_crs="EPSG:3644",
    dst_crs="EPSG:32610",
    left=bounds.left,
    right=bounds.right,
    bottom=bounds.bottom,
    top=bounds.top,
)

fig, axes = plt.subplots()
axes.set_aspect("equal")
image = axes.imshow(elevation, extent=extent)
fig.colorbar(image)

We only care about a small part of this domain, so next we'll pull out just the tile that we need.
I used [pyproj](https://pyproj4.github.io/pyproj) to roughly figure out a bounding box for the tile from the lat-lon coordinates given in the paper of 43.464${}^\circ$N, 124.119${}^\circ$W.
Then I eyeballed the limits of the domain from figure 5 in the paper.
We can then get the row and column indices in the DEM of the upper left and lower right corners to extract just the part of the data that we actually want to work with.

In [ ]:
(left, right), (top, bottom) = rasterio.warp.transform(
    src_crs="EPSG:3644",
    dst_crs="EPSG:32610",
    xs=[349750.0, 353350.0],
    ys=[647550.0, 644360.0],
)

rows, cols = rasterio.transform.rowcol(transform, (left, right), (top, bottom))
elevation = elevation[rows[0] : rows[1], cols[0] : cols[1]].astype(np.float64)

The slope that Roering 2008 used is the lobe that runs down the middle of the figure below, the ridge of which extends roughly along x = 409,400 m.

In [ ]:
fig, axes = plt.subplots()
image = axes.imshow(elevation, extent=(left, right, bottom, top))
fig.colorbar(image)

Next we'll use the [pysheds](https://mattbartos.com/pysheds/) package to calculate the catchment area throughout the domain.
The original results that we aim to reproduce here exclude any parts of the domain where the catchment area exceeds 250 m${}^2$, i.e. to capture the hillslopes and exclude valleys where fluvial transport dominates.

In [ ]:
import pysheds, pysheds.grid
import pyproj

crs = pyproj.Proj("epsg:32610")
window = rasterio.windows.Window(rows[0], cols[0], rows[1] - rows[0], cols[1] - cols[0])
transform = rasterio.windows.transform(window, transform)
grid = pysheds.grid.Grid(affine=transform, shape=elevation.shape, crs=crs)

The steps here are to (1) add the elevation data, (2) fill depressions that won't drain out of the domain, (3) remove flat parts of the DEM where a flow direction can't meaningfully be defined, and finally (4) calculate flow directions using the D${}^\infty$ routing algorithm from [Tarboton 1997](https://doi.org/10.1029/96WR03137).

In [ ]:
grid.add_gridded_data(elevation, "elevation", affine=transform, crs=crs)
grid.fill_depressions(data="elevation", out_name="flooded_elevation")
grid.resolve_flats(data="flooded_elevation", out_name="inflated_elevation")
grid.flowdir(data="inflated_elevation", out_name="flow_dir", routing="dinf")

Now that we've calculated the flow directions, we can calculate the accumulation or catchment areas.

In [ ]:
areas = grid.cell_area(as_crs=crs, inplace=False)
weights = (areas / areas.max()).ravel()
grid.accumulation(
    data="flow_dir", routing="dinf", weights=weights, out_name="accumulation"
)

The plot below shows the catchment area.
The dark blue areas are rivers and valleys, the bright yellow are ridge tops.

In [ ]:
from matplotlib.colors import LogNorm

fig, axes = plt.subplots(figsize=(8, 8))
norm = LogNorm(vmin=1, vmax=grid.accumulation.max() + 1)
image = axes.imshow(grid.accumulation + 1, extent=extent, cmap="viridis_r", norm=norm)
fig.colorbar(image)

This plot shows the elevation with the rivers and valleys masked out.

In [ ]:
maxval = 250.0
mask = grid.accumulation > maxval
elevation_masked = ma.masked_array(grid.elevation, mask=mask)
norm = LogNorm(vmin=1, vmax=maxval + 1)
fig, axes = plt.subplots(figsize=(8, 8))
image = axes.imshow(elevation_masked + 1, extent=extent)
fig.colorbar(image)

Next, we'll save the reprojected and windowed DEM and catchment areas to GeoTIFF files so that we can look at them in a GIS later.

In [ ]:
profile = {
    "driver": "GTiff",
    "count": 1,
    "height": elevation.shape[0],
    "width": elevation.shape[1],
    "crs": "EPSG:32610",
    "transform": transform,
    "dtype": np.float64,
}

with rasterio.open("elevation.tif", "w", **profile) as destination:
    destination.write(elevation, indexes=1)

with rasterio.open("catchment.tif", "w", **profile) as destination:
    destination.write(grid.accumulation, indexes=1)

Outside of the workflow in this notebook, I opened the catchment and DEM data sets in a GIS, manually traced the outline of a region that looks close enough to the domain in the paper, and saved that outline to a GeoJSON file.

In [ ]:
import geojson

with open("oregon-coast-range.geojson", "r") as outline_file:
    outline = geojson.load(outline_file)

The [icepack](https://icepack.github.io) package contains a utility routine called `collection_to_geo` that we'll use to convert the outline of the domain, stored as a GeoJSON feature collection, into the input format for [gmsh](https://gmsh.info), a program that generates unstructured triangular meshes.
The keyword argument `lcar` indicates the characteristic length scale for the mesh, which we've set to 5m.
You can tweak this value to get a finer or coarser mesh.

In [ ]:
import icepack

geometry = icepack.meshing.collection_to_geo(outline, lcar=5.0)
with open("oregon-coast-range.geo", "w") as geometry_file:
    geometry_file.write(geometry.get_code())

The .geo file that we've just created only describes the outline of the domain; to get a triangular mesh, we'll call gmsh from the command line.

In [ ]:
!gmsh -2 -format msh2 -v 2 -o oregon-coast-range.msh oregon-coast-range.geo

All the modeling in the following uses the [Firedrake](https://www.firedrakeproject.org) package.
Firedrake includes readers for meshes in several different formats, included that of gmsh.

In [ ]:
import firedrake

mesh = firedrake.Mesh("oregon-coast-range.msh")

The colors indicate the different boundary segments (see the legend).

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
firedrake.triplot(mesh, axes=axes)
axes.legend();

Before we start doing any modeling, we'll need to interpolate the gridded data that we read in above to some function space defined over this triangular mesh.
Here we're using the space of piecewise polynomials of degree 2 that are continuous across triangle boundaries; the `"CG"` stands for "Continuous Galerkin".

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2)

Firedrake doesn't know about raster data natively, so we'll use a utility function from icepack to interpolate the DEM to this function space.

In [ ]:
with rasterio.open("elevation.tif", "r") as dem_file:
    z = icepack.interpolate(dem_file, Q)

In [ ]:
fig, axes = plt.subplots()
colors = firedrake.tripcolor(z, axes=axes)
fig.colorbar(colors);